In [1]:
import requests as req

In [2]:
URL = 'https://www.colorado.gov/news'
res = req.get(URL)

In [3]:
res.content

b'<!DOCTYPE html>\n  <!--[if IEMobile 7]><html class="no-js ie iem7" lang="en" dir="ltr"><![endif]-->\n  <!--[if lte IE 6]><html class="no-js ie lt-ie9 lt-ie8 lt-ie7" lang="en" dir="ltr"><![endif]-->\n  <!--[if (IE 7)&(!IEMobile)]><html class="no-js ie lt-ie9 lt-ie8" lang="en" dir="ltr"><![endif]-->\n  <!--[if IE 8]><html class="no-js ie lt-ie9" lang="en" dir="ltr"><![endif]-->\n  <!--[if (gte IE 9)|(gt IEMobile 7)]><html class="no-js ie" lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema#"><![endif]-->\n  <!--[if !IE]><!--><html class="no-js" lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: ht

In [4]:
from bs4 import BeautifulSoup as bs

In [5]:
soup = bs(res.content, 'lxml')

In [6]:
divs = soup.find_all('div', {'class': 'view-page-item'})

In [10]:
divs[0].text

'\n\nWednesday, May 29, 2019\nGovernor Signs Nursing Home Penalty Cash Fund Bill\n\nFrom Department of Health Care Policy and Financing  \n'

In [11]:
len(divs)

10

In [12]:
soup

<!DOCTYPE html>
<!--[if IEMobile 7]><html class="no-js ie iem7" lang="en" dir="ltr"><![endif]--><!--[if lte IE 6]><html class="no-js ie lt-ie9 lt-ie8 lt-ie7" lang="en" dir="ltr"><![endif]--><!--[if (IE 7)&(!IEMobile)]><html class="no-js ie lt-ie9 lt-ie8" lang="en" dir="ltr"><![endif]--><!--[if IE 8]><html class="no-js ie lt-ie9" lang="en" dir="ltr"><![endif]--><!--[if (gte IE 9)|(gt IEMobile 7)]><html class="no-js ie" lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema#"><![endif]--><!--[if !IE]><!--><html class="no-js" dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: htt

In [13]:
# if you want to see what the scraped html looks like, you can save it like this, then open the file in a browser
with open('test.html', 'wb') as f:
    f.write(res.content)

# Prototype getting the date, title, link, and 'from' fields here

In [15]:
divs[0]

<div class="view-page-item">
<div class="date margin-b">
<span class="field-content"><span class="date-display-single" content="2019-05-29T13:04:17-06:00" datatype="xsd:dateTime" property="dc:date">Wednesday, May 29, 2019</span></span></div>
<h3 class="no-padding x-small lighter-text"><span class="field-content"><a href="https://www.colorado.gov/pacific/hcpf/news/governor-signs-nursing-home-penalty-cash-fund-bill">Governor Signs Nursing Home Penalty Cash Fund Bill</a></span></h3>
<div class="margin-b lighter-text smaller-text-views">
<span class="field-content">From Department of Health Care Policy and Financing  </span></div>
</div>

In [16]:
import pandas as pd

In [18]:
divs[0].find('span', {'class': 'date-display-single'}).text

'Wednesday, May 29, 2019'

In [17]:
pd.to_datetime(divs[0].find('span', {'class': 'date-display-single'}).text)

Timestamp('2019-05-29 00:00:00')

In [19]:
divs[0].find('a')

<a href="https://www.colorado.gov/pacific/hcpf/news/governor-signs-nursing-home-penalty-cash-fund-bill">Governor Signs Nursing Home Penalty Cash Fund Bill</a>

In [20]:
divs[0].find('a').text

'Governor Signs Nursing Home Penalty Cash Fund Bill'

In [21]:
divs[0].find('a').get('href')

'https://www.colorado.gov/pacific/hcpf/news/governor-signs-nursing-home-penalty-cash-fund-bill'

In [22]:
divs[0].find('div', {'class': 'smaller-text-views'}).text.strip()

'From Department of Health Care Policy and Financing'

In [23]:
PAGE_URL = 'https://www.colorado.gov/news?page={}'

In [25]:
import pandas as pd
from tqdm import tqdm_notebook
from pymongo import MongoClient

client = MongoClient()
db = client['news']
coll = db['colorado']

for page in tqdm_notebook(range(0, 66)):
#     for testing, we can stop it after a few iterations
#     if page == 3:
#         break      
    res = req.get(PAGE_URL.format(page))
    soup = bs(res.content, 'lxml')
    divs = soup.find_all('div', {'class': 'views-row'})
    for d in divs:
        date = pd.to_datetime(d.find('span', {'class': 'date-display-single'}).text)
        link = divs[0].find('a')
        title = link.text
        href = link.get('href')
        subtitle = d.find('div', {'class': 'smaller-text-views'}).text.strip()
        
        coll.insert_one({'date': date,
                         'title': title,
                         'link': href,
                         'subtitle': subtitle})

client.close()

HBox(children=(IntProgress(value=0, max=66), HTML(value='')))

In [26]:
from pprint import pprint

client = MongoClient()
db = client['news']
coll = db['colorado']

pprint(coll.find_one())

client.close()

{'_id': ObjectId('5cf81135ac76bc173d4b5710'),
 'date': datetime.datetime(2019, 5, 29, 0, 0),
 'link': 'https://www.colorado.gov/pacific/hcpf/news/governor-signs-nursing-home-penalty-cash-fund-bill',
 'subtitle': 'From Department of Health Care Policy and Financing',
 'title': 'Governor Signs Nursing Home Penalty Cash Fund Bill'}
